# LSA 이해를 위한 실습 코드
* 출처: [딥 러닝을 이용한 자연어 처리 입문 / 19-01 잠재 의미 분석 (Latent Semantic Analysis)](https://wikidocs.net/24949)
* 본 코드는 아래와 같은 순서로 구성되어 있습니다.
    1. Full SVD 생성
    2. 1에서 생성된 SVD를 기반으로 Truncated SVD 생성 (t=2)
    3. Truncated SVD의 의미 정리
    4. [응용] Scikit-learn 라이브러리의 Twenty Newsgroups 데이터를 이용하여 LSA 실습 (TF-IDF 행렬 생성)

## 1. Full SVD 생성

In [1]:
import numpy as np

![스크린샷 2023-06-13 143328](https://github.com/dev-hjJoo/NLP/assets/33647482/c793cb17-35dc-4631-a569-77f43dfcdd2f)
* 위 이미지와 같은 DTM을 생성한다.

In [2]:
A = np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
print('DTM의 크기(shape) :', np.shape(A))

DTM의 크기(shape) : (4, 9)


* 총 9개의 단어에 대한 문서 4개의 벡터를 생성하였으므로, Shape은 (4, 9)가 된다.
* 이에 대해서 Full SVD를 수행해본다.

### Full SVD를 이해하기 위한 용어
1. 직교 행렬(Orthogonal matrix): 자신과 전치 행렬의 곱(반대도 성립)의 결과가 단위 행렬이되는 행렬로, 직교 행렬의 역행렬은 전치행렬임.
2. 전치 행렬(Transpose matrix): 원래의 행렬에서 행과 열을 변환한 행렬
3. 단위 행렬(Identify matrix): 주대각선의 원소가 모두 1이며 원소는 모두 0인 **정사각** 행렬
4. 대각 행렬(Diagonal matrix): 주대각선을 제외한 곳의 원소가 모두 0인 행렬 (정사각 행렬이 아닐 수도 있음)

In [3]:
# U: 직교 행렬 (4*4)
# s: 특이값 벡터
# VT: V의 전치 행렬
U, s, VT = np.linalg.svd(A, full_matrices = True)
print('행렬 U :')
print(U.round(2))
print('행렬 U의 크기(shape) :',np.shape(U))

행렬 U :
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기(shape) : (4, 4)


In [4]:
print('특이값 벡터 :')
print(s.round(2))
print('특이값 벡터의 크기(shape) :',np.shape(s))

특이값 벡터 :
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기(shape) : (4,)


In [5]:
# 특이값 s는 대각행렬의 값만 추출한 것으로, 대각행렬을 확인하기 위해서는 크기가 알맞은 행렬 형태를 생성하여야 함
# 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S = np.zeros((4, 9))

# 특이값을 대각행렬에 삽입
S[:4, :4] = np.diag(s)

print('대각 행렬 S :')
print(S.round(2))

print('대각 행렬의 크기(shape) :')
print(np.shape(S))

대각 행렬 S :
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
대각 행렬의 크기(shape) :
(4, 9)


In [6]:
# V의 전치 행렬 VT 생성
print('직교행렬 VT :')
print(VT.round(2))

print('직교 행렬 VT의 크기(shape) :')
print(np.shape(VT))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교 행렬 VT의 크기(shape) :
(9, 9)


In [10]:
# U, S, VT를 곱하면 기존의 행렬 A가 나와야 함
# Numpy의 allclose() 메소드를 통해 행렬이 동일한지 확인

print('기존 행렬 A:')
print(A)
print('U, S, VT를 곱한 행렬:')
print(np.dot(np.dot(U, S), VT).round(2))    # 소수점 아래 두자리만 출력
print('두 행렬이 동일한 지 확인:', np.allclose(A, np.dot(np.dot(U, S), VT).round(2)))

기존 행렬 A:
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
U, S, VT를 곱한 행렬:
[[ 0.  0.  0.  1.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  1.  1. -0.  1. -0. -0.]
 [ 0.  1.  1. -0.  2. -0. -0.  0.  0.]
 [ 1. -0.  0.  0. -0.  0. -0.  1.  1.]]
두 행렬이 동일한 지 확인: True


---
## 2. Truncated SVD 생성
* 축소할 차원 수(토픽 수, t)는 2로 설정
* 여기서 차원 수는 상위 t개를 추출한다는 의미이며, t가 작을수록 원본 행렬과 차이가 커진다. (많이 축소시키기 때문)

In [11]:
# 특이값 상위 2개만 보존
S = S[:2,:2]

print('대각 행렬 S :')
print(S.round(2))

대각 행렬 S :
[[2.69 0.  ]
 [0.   2.05]]


In [12]:
# 직교 행렬 U에 대해서도 2개의 열만 남기고 제거
U = U[:,:2]
print('행렬 U :')
print(U.round(2))

행렬 U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [13]:
# VT에 대해서도 2개의 행만 남기고 제거 (V 관점에서 2개의 열만 남기고 제거한 것과 동일)
VT = VT[:2,:]
print('직교행렬 VT :')
print(VT.round(2))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [14]:
# 축소된 행렬에 대하여 U, S, VT를 곱하면 기존의 A와 다른 결과가 나옴 (값이 손실되었기 때문)
# 이렇게 나온 결과를 기존의 행렬 A와 비교
A_prime = np.dot(np.dot(U,S), VT)
print('기존 행렬 A:')
print(A)
print('차원축소 후 U, S, VT를 곱한 행렬:')
print(A_prime.round(2))

기존 행렬 A:
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
차원축소 후 U, S, VT를 곱한 행렬:
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


---
## 3. Truncated SVD의 의미 정리
* 기존 행렬 A의 값과 유사한 결과가 나오지만, 제대로 복구되지 않은 부분도 존재함.
#### 축소된 직교행렬 U 크기의 의미
* 축소된 U의 크기는 4\*2이며, 이는 문서의개수\*토픽의 개수를 의미한다.
* 즉, 총 4개의 문서에 대하여 단어 전체(9개)를 보지 않고 제한된 토픽(상위 2개)만 표현한 것이다.
* 제대로 복구되지 않은 것은 상위 n개에 속하지 않는 토픽으로 제거된 것으로 보아도 무관하다.

```최종적으로 Truncated SVD를 이용하여 데이터 정보를 압축하면서도 기존 행렬 A와 근사한 값을 가질 수 있음.```

---
## 4. Twenty Newsgroups 데이터를 이용한 LSA 실습
* LSA를 사용하여 문서의 수를 원하는 토픽의 수로 압축한 뒤, 각 토픽당 가장 중요한 단어5개를 출력한다.

#### 4-1. 뉴스 그룹 데이터에 대한 이해


In [17]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [18]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [19]:
# 훈련용 샘플 출력
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

* 뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어 문장이 포함되어 있음
* 위와 같은 형태의 샘플이 총 11,314개 존재

In [20]:
# 뉴스 그룹 데이터의 기존 카테고리 출력
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


#### 4-2. Preprocessing

In [21]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [22]:
# 정제 후 텍스트 확인
news_df['clean_doc'][1]

"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

In [24]:
# 불용어 데이터셋 다운로드
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\etoil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [25]:
# 불용어 처리
stop_words = stopwords.words('english') # NLTK로부터 불용어 불러오기
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거

In [26]:
# 불용어(your, about, just, that, will, after, ...) 제거된 텍스트 확인
print(tokenized_doc[1])

['yeah,', 'expect', 'people', 'read', 'faq,', 'etc.', 'actually', 'accept', 'hard', 'atheism?', 'need', 'little', 'leap', 'faith,', 'jimmy.', 'logic', 'runs', 'steam!', 'jim,', 'sorry', "can't", 'pity', 'you,', 'jim.', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well,', 'pretend', 'happily', 'ever', 'anyway.', 'maybe', 'start', 'newsgroup,', 'alt.atheist.hard,', "bummin'", 'much?', 'bye-bye,', 'jim.', 'forget', "flintstone's", 'chewables!', 'bake', 'timmons,']


#### 4-3. TF-IDF 행렬 만들기
* Scikit-learn의 TfidfVectorizer 사용
* 단어는 1,000개의 단어로 제한

In [28]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


#### 4-4. Topic modeling
* Scikit-learn의 TruncatedSVD 메소드 활용

In [29]:
# 기존의 뉴스그룹 데이터가 가진 카테고리 수가 20개이므로, n_components는 20으로 설정
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [31]:
# 전치 행렬 VT(=svd_model.components_)의 크기 출력: 토픽의 수 \* 단어의 수
np.shape(svd_model.components_)


(20, 1000)

* 아래 코드 실행할 때
    * sklearn의 버전이 0.24.x 이하인 경우 **get_feature_names()** 사용
    * sklearn의 버전이 1.0.x 이상인 경우 **get_feature_names_out()** 사용
    * 참고 링크: [[stack overflow] AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names_out'](https://stackoverflow.com/questions/70215049/attributeerror-tfidfvectorizer-object-has-no-attribute-get-feature-names-out)

In [33]:
# 20개의 행에 대하여 1000개의 열 중 가장 값이 큰 5개 출력
terms = vectorizer.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('just', 0.20273), ('don', 0.19949), ('like', 0.19534), ('know', 0.18819), ('people', 0.17851)]
Topic 2: [('thanks', 0.31973), ('windows', 0.27738), ('card', 0.17369), ('drive', 0.15927), ('mail', 0.1488)]
Topic 3: [('game', 0.31991), ('team', 0.27972), ('year', 0.26585), ('games', 0.20684), ('drive', 0.17141)]
Topic 4: [('edu', 0.42972), ('thanks', 0.24949), ('mail', 0.17243), ('game', 0.12812), ('team', 0.12645)]
Topic 5: [('know', 0.41918), ('does', 0.305), ('thanks', 0.26133), ('don', 0.21061), ('just', 0.19501)]
Topic 6: [('drive', 0.45668), ('edu', 0.21787), ('thanks', 0.18804), ('scsi', 0.1573), ('drives', 0.12178)]
Topic 7: [('just', 0.56807), ('edu', 0.43053), ('don', 0.22492), ('like', 0.19983), ('soon', 0.09527)]
Topic 8: [('chip', 0.2163), ('government', 0.2001), ('encryption', 0.14658), ('like', 0.14639), ('clipper', 0.14265)]
Topic 9: [('don', 0.32188), ('know', 0.31942), ('edu', 0.28677), ('does', 0.26522), ('think', 0.19896)]
Topic 10: [('does', 0.47651), ('ca